# Accelerating Python with Cython, Numba and JAX
> Scott Brandenberg, UCLA and Krishna Kumar, UT Austin

A Hands-On Tutorial
Wednesday, April 19, 2:00pm - 3:30pm US Central Time

The webinar will feature a hands-on component using DesignSafe Jupyter, so please be sure to register for a DesignSafe user account if you don't already have one. If your existing account is inactive, reactivate it by resetting your password.

Python is a popular programming language in natural hazards engineering research because it is free and open-source, and has a plethora of powerful packages for handling our community’s computing needs. However, Python is an interpreted language and is inherently slower and less efficient than compiled languages like Fortran, C, and C++. As a result, many scripts written in Python, particularly those involving loops, can run significantly faster with a few minor modifications. This webinar will demonstrate how vectorized calculations using Numpy arrays are significantly faster than the same operations coded in Python. We will demonstrate how to use Cython to compile Python code to C, which can improve performance by orders of magnitude. We will also demonstrate how to use just in time (JIT) compilation and JAX (Numpy on steroids) to accelerate Python, particularly using GPU’s.


In [1]:
%%html
<style>
table {float:left}
</style>

# Compiled Languages, Interpreted Languages, and JIT

To understand why execution speed is slower in Python than many other languages requires some background knowledge of compilers vs. interpreters, and compiled languages vs. interpreted languages. This section provides some background information.

## Compiled Language

A compiled language uses a compiler to translate source code to machine code (a.k.a. a binary executable file). The compiler reads the entire source code project to create an executable file specific to a particular hardware architecture. Examples of compiled languages are C, C++, and Fortran.  

<img src="https://raw.githubusercontent.com/kks32-courses/accelerating-python/main/images/CompiledLanguage.png" width="800">  
Figure 1. Schematic of compiled language operations.

## Interpreted Language

An interpreted language uses an interpreter to translate source code to machine code. The interpreter reads the source code line by line at runtime, and produces machine code on the fly. Examples of interpreted languages are Python, Javascript, and Ruby.

<img src="https://raw.githubusercontent.com/kks32-courses/accelerating-python/main/images/InterpretedLanguage.png" width="800">  
Figure 2. Schematic of interpreted language operations.

## Hybrid using Just In Time Compilation

Just in time (JIT) uses an interpreter to produce compiled code at runtime, and is a hybrid between a compiled language and an interpreted language. The first time the code is run may execute slowly, but subsequent runs will utilize the compiled machine code and will execute quickly.

<img src="https://raw.githubusercontent.com/kks32-courses/accelerating-python/main/images/HybridJIT.png" width="800">  
Figure 3. Schematic of just-in-time (JIT) compilation.

## Pros and Cons

Compiled languages have faster execution speed than interpreted languages because the compiler has already created the machine code. Interpeted languages have to perform runtime operations that reduce execution speed. For example, Python must check the type of each variable before a simple addition operation to make sure the types are compatible. If they aren't it returns an error. JIT may exhibit execution speeds comparable to compiled languages, with the caveat that the code must be compiled upon the first execution, which takes some time. Subsequent executions will be faster.

Interpreted languages are often faster for code development because many operations are handled by the interpreter, and need not be written by the code developer. For example, variable types do not need to be declared in Python because the interpeter infers them at rutime. This is very convenient, and allows developers to more rapidly write their code.

Compiled languages allow developers to keep their source code private because they can deploy a binary executable file that users can run on their own computers (assuming hardware compatibility). By contrast, interpreted languages require disclosure of the source code, which must be interpreted every time the code is executed.

Interpreted languages are more portable because the source code is disclosed, and can be interpreted on any user's computer. Compiled binary executables are hardware-specific, and different binary files may be required for different operating systems (e.g., PC vs. Mac vs. Unix).

Table 1. Pros and cons of compiled, interpreted, and hybrid (JIT).

|                  | Compiled | Interpreted | JIT |
| ---------------- | :------: | :---------: | :-: |
| Execution Speed  | ✔️       | ✖️         | ✔️* |
| Code Development | ✖️       | ✔️         | ✔️  |
| Private Code     | ✔️       | ✖️         | ✖️  |
| Portability      | ✖️       | ✔️         | ✔️  |  
|* JIT may have poor execution speed on the first run |

# Vectorized Calculation Using Numpy Arrays

Numpy is a the fundamental package for scientific computing in Python. Although Numpy is a Python package, it was not developed in Python. Rather, it is written mostly in C and consists of binary executables compiled from source code. Numpy functions are therefore generally significantly faster than the same operations performed in Python. The term "vectorized operation" refers to passing an entire Numpy array of known data type to an optimized, compiled C code. The example below shows a simple calculation of a harmonic function using vectorized operations compared with the same operation in a Python loop. The vectorized calculation is much faster.

In [4]:
import numpy as np
import time

# time step in seconds

# Number of time steps

# Frequency in Hz

# Numpy array containing time vector consisting of 32 bit floating point precision values

# Vectorized operation that passes time array into Numpy sin function

# Non-vectorized operation that uses a Python for loop

# Ratio of execution times


# Hands-on Cython and Numba

Python is a popular programming language in natural hazards engineering research because it is free and open-source, and has a plethora of powerful packages for handling our community’s computing needs. However, Python is an interpreted language and is inherently slower and less efficient than compiled languages like Fortran, C, and C++. As a result, many scripts written in Python, particularly those involving loops, can run significantly faster with a few minor modifications. This webinar will demonstrate how vectorized calculations using Numpy arrays and Scipy are significantly faster than the same operations coded in Python. We will demonstrate how to use Cython to compile Python code to C, which can improve performance by orders of magnitude. We will also demonstrate how to use just in time (JIT) compilation to accelerate Python, particularly using GPU’s.

## Heat Flow Problem

The relative performance of different coding approaches is demonstrated using a finite difference solution to the 2D transient heat flow problem for a square domain with a constant initial temperature subject to a temperature change on the top.

## Governing differential equation

$\frac{\partial T}{\partial t} = \alpha \left[ \frac{\partial ^2 T}{\partial x^2} + \frac{\partial^2 T}{\partial y^2}\right]$

T = temperature  
t = time  
x = horizontal dimension  
y = vertical dimension  
$\alpha$ = thermal diffusivity  

## Finite different approximation for a rectangular mesh
$\frac{\partial T}{\partial t} \approx \frac{T_{ij}^{k+1} - T_{ij}^{k}}{\Delta t}$  

$\frac{\partial ^2 T}{\partial x^2} \approx \alpha \frac{T_{i+1,j}^k -2T_{i,j}^k + T_{i-1,j}^k}{\Delta x^2}$  

$\frac{\partial ^2 T}{\partial y^2} \approx \alpha \frac{T_{1,j+1}^k -2T_{i,j}^k + T_{i,j-1}^k}{\Delta y^2}$  

$\Delta x$ = node spacing in x-direction  
$\Delta y$ = node spacing in y-direction  
$\Delta t$ = time step  
i = index counter for x-direction  
j = index counter for y-direction  
k = index counter for time  

### Resulting equation

$\frac{T_{ij}^{k+1} - T_{ij}^{k}}{\Delta t} = \alpha \frac{T_{i+1,j}^k -2T_{i,j}^k + T_{i-1,j}^k}{\Delta x^2} + \alpha \frac{T_{1,j+1}^k -2T_{i,j}^k + T_{i,j-1}^k}{\Delta y^2}$  

### If $\Delta x = \Delta y$ we obtain the following

$\frac{T_{ij}^{k+1} - T_{ij}^{k}}{\Delta t} = \alpha \frac{T_{i+1,j}^k + T_{1,j+1}^k -4T_{i,j}^k + T_{i-1,j}^k + T_{i,j-1}^k}{\Delta x^2}$  

### Solving for $T_{ij}^{k+1}$ and re-arranging terms

$T_{ij}^{k+1} = \gamma\left(T_{i+1,j}^k + T_{1,j+1}^k + T_{i-1,j}^k + T_{i,j-1}^k\right) + \left(1 - 4\gamma\right)T_{ij}^k$  

where $\gamma = \frac{\alpha \Delta t}{\Delta x^2}$


### Note: the solution will become unstable if $\left(1 - \frac{4\alpha \Delta t}{\Delta x^2} \right) < 0$. We therefore set the time step as shown below.

$\Delta t = \frac{\Delta x^2}{4\alpha}$  

### Using the time step above, we find that $\left(1-4\gamma\right)=0$ and therefore the resulting equation is:

$T_{ij}^{k+1} = \gamma\left(T_{i+1,j}^k + T_{i,j+1}^k + T_{i-1,j}^k + T_{i,j-1}^k\right)$  

## Define input variables

`L`         = plate length  
`Nt`        = number of time steps  
`Nx`        = number of increments in x-direction (same as y-direction since plate is square)  
`alpha`     = thermal diffusivity  
`dx`        = node spacing  
`dt`        = time increment  
`T_top`     = temperature of top of plate  
`T_left`    = temperature of left side of plate  
`T_right`   = temperature of right side of plate  
`T_bottom`  = temperature of bottom of plate  
`T_initial` = initial temperature of plate  

In [6]:
import numpy as np
import matplotlib.pyplot as plt

L = 50
Nt = 1000
Nx = 50
alpha = 2.0
dx = L/Nx
dt = dx**2/4.0/alpha
gamma = alpha*dt/dx/dx
T_top = 100.0
T_left = 0.0
T_right = 0.0
T_bottom = 0.0
T_initial = 0.0

# Initialize Numpy array T to store temperature values
T = np.full((Nt,Nx,Nx),T_initial,dtype=float)
T[:,:,:1] = T_left
T[:,:,Nx-1] = T_right
T[:,:1,:] = T_bottom
T[:,Nx-1:, :] = T_top


## Implementation Using Python Loops

The function below utilizes nested Python loops to define the temperature array. It accepts an initialized temperature array $T$ and $\gamma$ as arguments and returns the updated temperature array. 

In [7]:
def calculate_python(T,gamma):
    Nt = len(T)
    Nx = len(T[0])
    for k in range(0,Nt-1,1):
        for i in range(1,Nx-1,1):
            for j in range(1,Nx-1,1):
                T[k+1,i,j] = gamma*(T[k,i+1,j] + T[k,i-1,j] + T[k,i,j+1] + T[k,i,j-1])
    return T

## Implementation Using Cython

Cython is an optimising static compiler for both the Python programming language and the extended Cython programming language (based on Pyrex). It makes writing C extensions for Python as easy as Python itself. 

https://cython.org/

### Dependencies

* C compiler
* Cython package
* Python

Cython can be implemented in different ways. One way is to create a Cython file with a .pyx extension, and write a separate setup.py file that instructs the compiler how to compile your Cython code to C. Another way is to utilize the Cython magic within Jupyter, which is what we will do here.

### Additional information

To indicate you want to run a cython cell, include %%cython at the top of your notebook cell.

To include hints about native Python code that might slow things down, use %%cython -a. Your goal should be to remove as many yellow highlights as possible, especially inside loops. Yellow highlights indicate Python overhead where the work of the Python interpreter will be compiled into your C code, thereby making it slow.

This example will use typed memoryviews to handle NumPy arrays. https://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html

In [8]:
# install the Cython package
!pip install --user cython

# load the Cython package
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [9]:
%%cython
import cython
import numpy as np
cimport numpy as np


# Implementation Using Numba JIT

Numba provides several utilities for code generation, but its central feature is the numba.jit() decorator. Using this decorator, you can mark a function for optimization by Numba’s JIT compiler. Various invocation modes trigger differing compilation options and behaviours.

https://numba.pydata.org/numba-doc/latest/user/jit.html

In [10]:
!pip install --user numba

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 19.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 20.4 MB/s eta 0:00:0000:0100:01


In [11]:
import numba
import numpy as np
from numba.typed import List

# Using Scipy's Convolve Operator

We know that Python loops are slow thanks to the interpreter. We also know that we can use Cython and Numba / JIT to significantly improve performance. It turns out that we can also use the SciPy signal package to run this code efficiently using the convolve operator. This is kind of equivalent to vectorized calculations because we are passing the loops to efficiently compiled C code within SciPy instead of performing those loops in Python.

https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve.html

![vectorization](https://raw.githubusercontent.com/kks32-courses/accelerating-python/main/images/convolution.png)

In [12]:
import numpy as np
from scipy.signal import convolve

In [ ]:
## Performance Measurement Summary

import time
start_time = time.time()
T_python = calculate_python(T, gamma)
print(f"Pure Python time: = {time.time() - start_time:.3f} seconds")
start_time = time.time()
T_cython = calculate_cython(T, gamma)
print(f"Cython time: = {time.time() - start_time:.3f} seconds")
start_time = time.time()
T_numba = calculate_numba_wrapper(T, gamma)
print(f"Numba time (first run): = {time.time() - start_time:.3f} seconds")
start_time = time.time()
T_numba = calculate_numba_wrapper(T, gamma)
print(f"Numba time (second run): = {time.time() - start_time:.3f} seconds")
start_time = time.time()
T_convolve = calculate_conv(T, gamma)
print(f"Python Convolution time: = {time.time() - start_time:.3f} seconds")

## Plot heat maps to make sure the algorithms are the same

In [ ]:
fig, ax = plt.subplots(ncols=4,figsize=(12,3),sharey='row')
k = 999
data = {'Python':T_python[k], 'Cython':T_cython[k], 'Numba':T_numba[k], 'Convolve':T_convolve[k]}
i = 0
for key, value in data.items():
    pcm = ax[i].pcolormesh(value, cmap=plt.cm.viridis, vmin=0, vmax=100)
    ax[i].set_xlabel('x-position')
    ax[i].set_aspect('equal')
    ax[i].annotate(key, xy=(1,1), c='white', fontsize=15)
    fig.colorbar(pcm,ax=ax[i],shrink=0.75)
    i+=1    
ax[0].set_ylabel('y-position')
#fig.colorbar(pcm)
plt.tight_layout()

## Large-scale problem

Cython and Numba are so much faster than pure Python. They are in fact so fast that it's difficult to observe differences in performance for the small domain we solved previously. So let's increase the mesh density so the calculations take a little bit longer

In [ ]:
L = 50
Nt = 4000
Nx = 200
alpha = 2.0
dx = L/Nx
dt = dx**2/4.0/alpha
T_top = 100.0
T_left = 0.0
T_right = 0.0
T_bottom = 0.0
T_initial = 0.0
T = np.empty((Nt,Nx,Nx))
T.fill(T_initial)
T[:,:,:1] = T_left
T[:,:,Nx-1] = T_right
T[:,:1,:] = T_bottom
T[:,Nx-1:, :] = T_top
start_time = time.time()
T_cython = calculate_cython(T, gamma)
print(f"Cython time: = {time.time() - start_time:.3f} seconds")
start_time = time.time()
T_numba = calculate_numba_wrapper(T, gamma)
print(f"Numba time: = {time.time() - start_time:.3f} seconds")